In [ ]:
!pip install keras-tuner -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.3 MB/s eta 0:00:00


In [ ]:
import keras
from keras import layers
import keras_tuner

def model_built(hp):
  model = keras.Sequential()
  model.add(layers.Flatten())
  # we will do hyperperemeter in first layer only
  model.add(layers.Dense(
      units = hp.Int("units", min_value = 10, max_value = 512, step =30),
      activation = hp.Choice("activation", ["relu", "softmax", "tanh"])
    )
  )
  # tune wheather to use dropout
  if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=0.25))

  model.add(layers.Dense(10, activation="softmax"))
  # here we use learning rate
  learning_rate = hp.Float("lr", min_value = 1e-4, max_value = 1e-2, sampling="log")

  model.compile(
        optimizer =keras.optimizers.Adam(learning_rate=learning_rate),
        loss = "categorical_crossentropy",
        metrics = ["accuracy"]
     )
  return model

model_built(keras_tuner.HyperParameters())

# lets see what is 'hp.Int' function
hp = keras_tuner.HyperParameters()
print(hp.Int("units", min_value = 10, max_value = 512, step =30))

10


In [ ]:
# We can also create hyperparameter and kares code in seperate

def keras_model(units, activation, dropout, lr):
  model = keras.Sequential()
  model.add(layers.Flatten())
  model.add(layers.Dense(
      units = units,
      activation = activation,
  ))
  model.add(layers.Dropout(rate=0.25))
  model.add(layers.Dense(10, activation='relu'))

  model.compile(
      optimizer = keras.optimizers.Adam(learning_rate=lr),
      loss = "catagorical_crossentropy",
      metrics = ["accuracy"],
  )

  return model

# here we seperate the hyperparameter
def built_model(hp):
  units = hp.Int("units", min_value=10, max_value=512, step = 32),
  activation = hp.Choice("activation", ["relu","softmax","tanh"])
  dropout = hp.Boolean('dropout')
  lr = hp.Float("lr", min_value = 1e-4, max_value=1e-2, sampling="log")

  model = keras_model(
      units=units,
      activation=activation,
      dropout=0.25,
      lr=lr)
  return model

built_model(keras_tuner.HyperParameters())

<Sequential name=sequential_9, built=False>

In [ ]:
# here our main aim is to tune the number of hidden layer
def build_model_with_tunning_layers(hp):
    model = keras.Sequential()
    model.add(layers.Flatten())

    # Tune the number of layers.
    for i in range(hp.Int("total_layer", 1, 25)):
      model.add(
          layers.Dense(
              units= hp.Int(f"unit_{i}", min_value=10, max_value=512, step=30),
              activation = hp.Choice("activation", ["relu", "softmax","tanh"])
          )
      )

    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation="softmax"))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


build_model_with_tunning_layers(keras_tuner.HyperParameters())

<Sequential name=sequential_13, built=False>

In [ ]:
# start the search
tuner = keras_tuner.RandomSearch(
    hypermodel = build_model_with_tunning_layers,
    objective = "val_accuracy",
    max_trials= 2,
    executions_per_trial = 5,
    overwrite = "True",
    directory = " my_dict",
    project_name = "hello world"
)

tuner.search_space_summary()

Search space summary
Default search space size: 5
total_layer (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 25, 'step': 1, 'sampling': 'linear'}
unit_0 (Int)
{'default': None, 'conditions': [], 'min_value': 10, 'max_value': 512, 'step': 30, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'softmax', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


Prepare the dataset first


In [ ]:
import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 2 Complete [00h 02m 57s]
val_accuracy: 0.9622200131416321

Best val_accuracy So Far: 0.9622200131416321
Total elapsed time: 00h 06m 45s


In [ ]:
#  show the output of best model
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 70)                  │          54,950 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 40)                  │           2,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 460)                 │          18,860 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 340)                 │         156,740 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 10)                  │           3,410 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 236,800 (925.00 KB)

 Trainable params: 236,800 (925.00 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
tuner.results_summary()

Results summary
Results in  my_dict/hello world
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
total_layer: 4
unit_0: 70
activation: relu
dropout: False
lr: 0.001024143840053108
unit_1: 40
unit_2: 460
unit_3: 340
unit_4: 310
unit_5: 490
unit_6: 340
unit_7: 400
unit_8: 40
unit_9: 160
unit_10: 130
unit_11: 310
unit_12: 40
unit_13: 490
unit_14: 100
unit_15: 370
unit_16: 280
unit_17: 130
Score: 0.9622200131416321

Trial 0 summary
Hyperparameters:
total_layer: 18
unit_0: 310
activation: relu
dropout: True
lr: 0.0007359237843776363
unit_1: 10
unit_2: 10
unit_3: 10
unit_4: 10
unit_5: 10
unit_6: 10
unit_7: 10
unit_8: 10
unit_9: 10
unit_10: 10
unit_11: 10
unit_12: 10
unit_13: 10
unit_14: 10
unit_15: 10
unit_16: 10
unit_17: 10
Score: 0.4342000037431717


In [ ]:
# retrain the model

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((x_train, x_val))
y_all = np.concatenate((y_train, y_val))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 15s 7ms/step - accuracy: 0.8703 - loss: 0.4145
